In [1]:
from Model_architecture.modelArch import DenseArchs
import cv2
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import numpy as np
import os
import pandas as pd



In [2]:
def get_pixel(img, center, x, y):
    	
	new_value = 0
	
	try:
		# If local neighbourhood pixel
		# value is greater than or equal
		# to center pixel values then
		# set it to 1
		if img[x][y] >= center:
			new_value = 1
			
	except:
		# Exception is required when
		# neighbourhood value of a center
		# pixel value is null i.e. values
		# present at boundaries.
		pass
	
	return new_value

# Function for calculating LBP
def lbp_calculated_pixel(img, x, y):

	center = img[x][y]

	val_ar = []
	
	# top_left
	val_ar.append(get_pixel(img, center, x-1, y-1))
	
	# top
	val_ar.append(get_pixel(img, center, x-1, y))
	
	# top_right
	val_ar.append(get_pixel(img, center, x-1, y + 1))
	
	# right
	val_ar.append(get_pixel(img, center, x, y + 1))
	
	# bottom_right
	val_ar.append(get_pixel(img, center, x + 1, y + 1))
	
	# bottom
	val_ar.append(get_pixel(img, center, x + 1, y))
	
	# bottom_left
	val_ar.append(get_pixel(img, center, x + 1, y-1))
	
	# left
	val_ar.append(get_pixel(img, center, x, y-1))
	
	# Now, we need to convert binary
	# values to decimal
	power_val = [1, 2, 4, 8, 16, 32, 64, 128]

	val = 0
	
	for i in range(len(val_ar)):
		val += val_ar[i] * power_val[i]
		
	return val


In [3]:

n_classes = 2
arc = DenseArchs(n_classes)

face_model = arc.arch()
face_model.summary()
    

training initiated
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 224, 224, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 112, 112, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 112, 112, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 56, 56, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 200704)            0         
                                                                 
 dense (Dense)               (None,

In [4]:
x_data=[]
y_data = []
learning_rate = 0.01
epoch = 100
batch_size = 10          
people = sorted(os.listdir('people'))

for x in people:
    for i in os.listdir('people/'+x):
        img1 = cv2.imread('people'+'/'+x+'/'+i, -1)
        img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
        img_lbp = np.zeros(img.shape[:2],np.uint8)
        for i in range(0, img.shape[0]):
            for j in range(0, img.shape[1]):
                img_lbp[i, j] = lbp_calculated_pixel(img, i, j)
        img = cv2.resize(img_lbp, (224, 224)) 
        img = img.astype('float')/255.0
        img = np.expand_dims(img, axis=0)
        img = np.reshape(img,[224,224])
        x_data.append(img)
        y_data.append(int(x[0:x.find('_')])-1)
print(y_data)
x_data = np.array(x_data, dtype='float')
y_data = np.array(y_data)
y_data = y_data.reshape(len(y_data), 1)
    
X = x_data
y = to_categorical(y_data, num_classes=n_classes)

    # o = Adam(decay=learning_rate/epoch)
face_model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])

face_model.fit(X, y=y, batch_size=batch_size, validation_split=0.1,epochs=epoch,verbose=2)
face_model.save('LBPCNNModel/face_recog_model.h5') 
print("Training Completed")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Epoch 1/100
6/6 - 4s - loss: 47.2496 - accuracy: 0.4259 - val_loss: 0.7906 - val_accuracy: 0.0000e+00 - 4s/epoch - 668ms/step
Epoch 2/100
6/6 - 0s - loss: 0.7039 - accuracy: 0.5185 - val_loss: 0.7500 - val_accuracy: 0.0000e+00 - 124ms/epoch - 21ms/step
Epoch 3/100
6/6 - 0s - loss: 0.6224 - accuracy: 0.7963 - val_loss: 0.5090 - val_accuracy: 1.0000 - 121ms/epoch - 20ms/step
Epoch 4/100
6/6 - 0s - loss: 0.6140 - accuracy: 0.7963 - val_loss: 0.0011 - val_accuracy: 1.0000 - 120ms/epoch - 20ms/step
Epoch 5/100
6/6 - 0s - loss: 0.2325 - accuracy: 0.9259 - val_loss: 0.0091 - val_accuracy: 1.0000 - 121ms/epoch - 20ms/step
Epoch 6/100
6/6 - 0s - loss: 0.0105 - accuracy: 1.0000 - val_loss: 0.0037 - val_accuracy: 1.0000 - 121ms/epoch - 20ms/step
Epoch 7/100
6/6 - 0s - loss: 0.0018 - accuracy: 1.0000 - val_loss: 7.95

Epoch 63/100
6/6 - 0s - loss: 9.4042e-07 - accuracy: 1.0000 - val_loss: 5.1090e-08 - val_accuracy: 1.0000 - 121ms/epoch - 20ms/step
Epoch 64/100
6/6 - 0s - loss: 9.2055e-07 - accuracy: 1.0000 - val_loss: 5.1090e-08 - val_accuracy: 1.0000 - 123ms/epoch - 21ms/step
Epoch 65/100
6/6 - 0s - loss: 9.0069e-07 - accuracy: 1.0000 - val_loss: 5.1090e-08 - val_accuracy: 1.0000 - 123ms/epoch - 20ms/step
Epoch 66/100
6/6 - 0s - loss: 8.7861e-07 - accuracy: 1.0000 - val_loss: 5.1090e-08 - val_accuracy: 1.0000 - 123ms/epoch - 21ms/step
Epoch 67/100
6/6 - 0s - loss: 8.6316e-07 - accuracy: 1.0000 - val_loss: 5.1090e-08 - val_accuracy: 1.0000 - 123ms/epoch - 20ms/step
Epoch 68/100
6/6 - 0s - loss: 8.4770e-07 - accuracy: 1.0000 - val_loss: 5.1090e-08 - val_accuracy: 1.0000 - 124ms/epoch - 21ms/step
Epoch 69/100
6/6 - 0s - loss: 8.3004e-07 - accuracy: 1.0000 - val_loss: 3.4060e-08 - val_accuracy: 1.0000 - 124ms/epoch - 21ms/step
Epoch 70/100
6/6 - 0s - loss: 8.1459e-07 - accuracy: 1.0000 - val_loss: 3.40

In [5]:
y

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [0., 1.

# Testing

In [6]:
from tabnanny import verbose
from unicodedata import decimal
import cv2
from FaceDetection.face_detection import face
from keras.models import load_model
import numpy as np
import os

import warnings
warnings.filterwarnings("ignore")

In [7]:
label = None

person = None

def Create_labels():
    people = sorted(os.listdir('people'))
    students = {}
    attendance_count = {}
    for i in people:
        students[(int(i[0:i.find('_')])-1)]=i[i.find('_')+1:]
        attendance_count[(int(i[0:i.find('_')])-1)] = 0
    return students

In [8]:
Create_labels()
people = sorted(os.listdir('peeps'))
model = load_model('LBPCNNModel/face_recog_model.h5')
# print(people)
names={0:'Akshansh',1:'Vivek'}
names_i={'A':'Akshansh','V':'Vivek'}

result=[]
peeps=[]
total=0
for x in people:
    for i in os.listdir('peeps/'+x):
        img1 = cv2.imread('peeps'+'/'+x+'/'+i, -1)
        img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
        img_lbp = np.zeros(img.shape[:2],np.uint8)
        total=total+1
        i=i[0:i.find("_")]
        print(x,i)
        
        peeps.append(i)
        for i in range(0, img.shape[0]):
            for j in range(0, img.shape[1]):
                img_lbp[i, j] = lbp_calculated_pixel(img, i, j)
        img = cv2.resize(img_lbp, (224, 224))
        img = img.astype('float')/255.0
        img = np.expand_dims(img, axis=0)
        prediction = model.predict(img,verbose=2)
        
        result.append(int(np.argmax(prediction)))
        
#         print(prediction)
#         print(int(np.argmax(prediction)))
        

all A
1/1 - 0s - 125ms/epoch - 125ms/step
all A
1/1 - 0s - 13ms/epoch - 13ms/step
all A
1/1 - 0s - 16ms/epoch - 16ms/step
all A
1/1 - 0s - 12ms/epoch - 12ms/step
all A
1/1 - 0s - 13ms/epoch - 13ms/step
all A
1/1 - 0s - 14ms/epoch - 14ms/step
all A
1/1 - 0s - 14ms/epoch - 14ms/step
all A
1/1 - 0s - 15ms/epoch - 15ms/step
all A
1/1 - 0s - 14ms/epoch - 14ms/step
all A
1/1 - 0s - 15ms/epoch - 15ms/step
all A
1/1 - 0s - 14ms/epoch - 14ms/step
all V
1/1 - 0s - 15ms/epoch - 15ms/step
all V
1/1 - 0s - 15ms/epoch - 15ms/step
all V
1/1 - 0s - 13ms/epoch - 13ms/step
all V
1/1 - 0s - 15ms/epoch - 15ms/step
all V
1/1 - 0s - 15ms/epoch - 15ms/step
all V
1/1 - 0s - 16ms/epoch - 16ms/step
all V
1/1 - 0s - 14ms/epoch - 14ms/step
all V
1/1 - 0s - 13ms/epoch - 13ms/step


In [11]:
y=[]
yp=[]
for ypi,yi in zip(result,peeps):
    print(names_i[yi],end=" ")
    print(names[ypi])
    y.append(names_i[yi])
    yp.append(names[ypi])

Akshansh Akshansh
Akshansh Akshansh
Akshansh Akshansh
Akshansh Akshansh
Akshansh Akshansh
Akshansh Akshansh
Akshansh Akshansh
Akshansh Akshansh
Akshansh Akshansh
Akshansh Akshansh
Akshansh Akshansh
Vivek Vivek
Vivek Vivek
Vivek Vivek
Vivek Vivek
Vivek Vivek
Vivek Vivek
Vivek Vivek
Vivek Vivek


In [12]:
from sklearn.metrics import confusion_matrix
import matplotlib as plt


conf_matrix=confusion_matrix(y, yp, labels=["Akshansh","Vivek"])

print("Confusion Matrix: \n")
print(conf_matrix)
num=0
for i in range(2):
    num=num+conf_matrix[i][i]
print("\nAccuracy on test data: ",num/total*100)

Confusion Matrix: 

[[11  0]
 [ 0  8]]

Accuracy on test data:  100.0
